In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
df=pd.read_csv('../input/home-data-for-ml-course/train.csv')
feature=['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X=df[feature].copy()
y=df.SalePrice.copy()

In [4]:
X.shape

(1460, 7)

In [5]:
y.shape

(1460,)

In [6]:
X.isnull().sum()

LotArea         0
YearBuilt       0
1stFlrSF        0
2ndFlrSF        0
FullBath        0
BedroomAbvGr    0
TotRmsAbvGrd    0
dtype: int64

In [7]:
X.describe()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,1971.267808,1162.626712,346.992466,1.565068,2.866438,6.517808
std,9981.264932,30.202904,386.587738,436.528436,0.550916,0.815778,1.625393
min,1300.000000,1872.000000,334.000000,0.000000,0.000000,0.000000,2.000000
25%,7553.500000,1954.000000,882.000000,0.000000,1.000000,2.000000,5.000000
50%,9478.500000,1973.000000,1087.000000,0.000000,2.000000,3.000000,6.000000
75%,11601.500000,2000.000000,1391.250000,728.000000,2.000000,3.000000,7.000000
max,215245.000000,2010.000000,4692.000000,2065.000000,3.000000,8.000000,14.000000


In [8]:
X_tr,X_t,y_tr,y_t=train_test_split(X,y,test_size=0.3,random_state=1)

In [9]:
model=DecisionTreeRegressor(random_state=1)

In [10]:
model.fit(X_tr,y_tr)
prediction=model.predict(X_t)
mae=mean_absolute_error(prediction,y_t)
mae

31392.527397260274

In [11]:
def model_select_node(Xtr,Xt,ytr,yt,node,rand):
    model=DecisionTreeRegressor(max_leaf_nodes=node,random_state=rand)
    model.fit(Xtr,ytr)
    pred=model.predict(Xt)
    mae=mean_absolute_error(pred,yt)
    return mae



In [12]:
node=range(2,1000)
score={node:model_select_node(X_tr,X_t,y_tr,y_t,node,1) for node in node}
best_score=min(score,key=score.get)
print("the best size of node is {} and has score {}".format(best_score,score[best_score]))

the best size of node is 43 and has score 28524.28455264335


In [13]:
model_random=RandomForestRegressor(random_state=1)
model_random.fit(X_tr,y_tr)
prediction=model_random.predict(X_t)
mae=mean_absolute_error(prediction,y_t)
mae

23091.682766905844

In [14]:
model_final=RandomForestRegressor(random_state=1)
model_final.fit(X,y)

RandomForestRegressor(random_state=1)

In [15]:
test=pd.read_csv('../input/home-data-for-ml-course/test.csv')

In [16]:
test_data=test[feature]
prediction=model_final.predict(test_data)
prediction

array([122656.58, 156789.  , 182959.  , ..., 151283.01, 127878.  ,
       225959.8 ])

In [17]:
data_sale=pd.DataFrame({'Id':test.Id,'SalePrice':prediction})

In [18]:
data_sale

,Id,SalePrice
0,1461,122656.58
1,1462,156789.00
2,1463,182959.00
3,1464,178102.00
4,1465,189049.48
...,...,...
1454,2915,83645.00
1455,2916,86785.00
1456,2917,151283.01
1457,2918,127878.00


In [19]:
data_finish=test.merge(data_sale,on='Id',how='left')
data_finish

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,122656.58
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,156789.00
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,182959.00
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,178102.00
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,189049.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,83645.00
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,86785.00
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,151283.01
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,127878.00


In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
model_1=RandomForestRegressor(n_estimators=50,random_state=0)
model_2=RandomForestRegressor(n_estimators=100,random_state=0)
model_3=RandomForestRegressor(n_estimators=100,criterion='mae',random_state=0)
model_4=RandomForestRegressor(n_estimators=200,min_impurity_split=20,random_state=0)
model_5=RandomForestRegressor(n_estimators=100,max_depth=7,random_state=0)
models=[model_1,model_2,model_3,model_4,model_5]
def model_selection(model,a=X_tr,b=X_t,c=y_tr,d=y_t):
    model.fit(X_tr,y_tr)
    preds=model.predict(X_t)
    return mean_absolute_error(preds,y_t)
for i in range(0,len(models)):
    mae=model_selection(models[i])
    print('Model {} MAE {}'.format(i+1,round(mae,2)))
    


Model 1 MAE 23085.54
Model 2 MAE 23047.98
Model 3 MAE 23221.4
Model 4 MAE 23115.04
Model 5 MAE 23526.01
